In [1]:
import numpy as np
import py3Dmol as p3d
import veloxchem as vlx

In [2]:
ch4_ff = vlx.Molecule.smiles_to_xyz("C")
h2o_ff = vlx.Molecule.smiles_to_xyz("O")
co2_ff = vlx.Molecule.smiles_to_xyz("O=C=O")
o2_ff = vlx.Molecule.smiles_to_xyz("O=O")

In [3]:
def xtb_optimization(xyz_ff):

    molecule = vlx.Molecule.read_xyz_string(xyz_ff)
    xtb_drv = vlx.XtbDriver()
    
    xtb_opt_drv = vlx.OptimizationDriver(xtb_drv)
    xtb_opt_drv.ostream.mute()
    
    xtb_opt = xtb_opt_drv.compute(molecule)
    
    return xtb_opt["final_geometry"]


ch4_xtb = xtb_optimization(ch4_ff)
h2o_xtb = xtb_optimization(h2o_ff)
co2_xtb = xtb_optimization(co2_ff)
o2_xtb = xtb_optimization(o2_ff)

In [4]:
viewer = p3d.view(viewergrid=(2, 2), width=400, height=300, linked=False)
viewer.addModel(ch4_xtb, "xyz", viewer=(0, 0))
viewer.addModel(o2_xtb, "xyz", viewer=(0, 1))
viewer.addModel(co2_xtb, "xyz", viewer=(1, 0))
viewer.addModel(h2o_xtb, "xyz", viewer=(1, 1))
viewer.setViewStyle({"style": "outline", "width": 0.05})
viewer.setStyle({"stick": {}, "sphere": {"scale": 0.25}})
viewer.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [5]:
def scf_calculation(molecule_xyz, basis_set, xc, multiplicity=None):

    molecule = vlx.Molecule.read_xyz_string(molecule_xyz)
    basis = vlx.MolecularBasis.read(molecule, basis_set, ostream=None)

    if multiplicity is not None:
        molecule.set_multiplicity(multiplicity)
        molecule.check_multiplicity()
        scf_gs = vlx.ScfUnrestrictedDriver()
        scf_gs.ostream.mute()
    else:
        scf_gs = vlx.ScfRestrictedDriver()
        scf_gs.ostream.mute()

    scf_gs.xcfun = xc
    scf_results = scf_gs.compute(molecule, basis)

    return scf_results, scf_gs


xc = "b3lyp"
basis_set = "6-311G*"

In [6]:
ch4_results, ch4_scf = scf_calculation(ch4_xtb, basis_set, xc)
o2_results, o2_scf = scf_calculation(o2_xtb, basis_set, xc, multiplicity=3)
co2_results, co2_scf = scf_calculation(co2_xtb, basis_set, xc)
h2o_results, h2o_scf = scf_calculation(h2o_xtb, basis_set, xc)

In [7]:
e_reactants = ch4_scf.get_scf_energy() + 2 * o2_scf.get_scf_energy()
e_products = co2_scf.get_scf_energy() + 2 * h2o_scf.get_scf_energy()

print(f"The reaction energy is {627.5*(e_products - e_reactants):.1f} kcal/mol")

The reaction energy is -157.1 kcal/mol
